# Maximum Likelihood Amplitude Estimation (MLAE) module

The present notebook provides an overview of the **Maximum Likelihood Amplitude Estimation (MLAE)** algorithm, which has been implemented in the `maximum_likelihood_ae` module of the **AE** package within the *QQuantLib* library (**QQuantLib/AE/maximum_likelihood_ae.py**). 

Within this module, the **MLAE** algorithm is encapsulated as a Python class, allowing for modular and reusable implementation. This approach facilitates its integration into larger quantum computing workflows and applications.

The content of this notebook and the associated module are based on the following references:

- **Suzuki, Y., Uno, S., Raymond, R., Tanaka, T., Onodera, T., & Yamamoto, N.**  
  *Amplitude Estimation without Phase Estimation.*  
  Quantum Information Processing, 19(2), 2020.  
  [https://arxiv.org/abs/1904.10246](https://arxiv.org/abs/1904.10246)

- **NEASQC Deliverable: D5.1**  
  *Review of State-of-the-Art for Pricing and Computation of VaR.*  
  [https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf](https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf)

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]

linalg_qpu = get_qpu(my_qpus[1])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

In [ ]:
#For transform bits to int
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before performing any amplitude estimation, we first need to load data into the quantum circuit. As this step is auxiliary and intended to demonstrate how the algorithm works, we will simply load a discrete probability distribution. 

In this example, we will use a quantum circuit with $ n = 3 $ qubits, which corresponds to a total of $ N = 2^n = 8 $ computational basis states. The discrete probability distribution we aim to load is defined as:

$$
p_d = \frac{(0, 1, 2, 3, 4, 5, 6, 7)}{0 + 1 + 2 + 3 + 4 + 5 + 6 + 7}.
$$

This distribution assigns probabilities proportional to the integers $ 0 $ through $ 7 $, normalized by their sum to ensure that the total probability equals 1.

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function `load_probability` from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
oracle = load_probability(probability)

In [ ]:
%qatdisplay oracle --svg

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. MLAE algorithm step by step.

The problem of amplitude estimation can be formulated as follows. Given an oracle operator $\mathcal{A}$, we have:

$$
\mathcal{A}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle + \sqrt{1-a}|\Psi_1\rangle,
$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states. The goal is to estimate the value of $\sqrt{a}$. To simplify the problem, we associate an angle $\theta$ with $\sqrt{a}$ such that $\sin^2{\theta} = a$. Consequently, the state $|\Psi\rangle$ can be rewritten as:

$$
\mathcal{A}|0\rangle = |\Psi\rangle = \sin(\theta)|\Psi_0\rangle + \cos(\theta)|\Psi_1\rangle. \tag{1}
$$

We have implemented a Python class named `MLAE` in the **QQuantLib/AE/maximum_likelihood_ae** module, which provides tools to implement the **MLAE** algorithm. In this section, we will describe the structure of the class step by step and provide an overview of the fundamental principles behind the **MLAE** algorithm.

In [ ]:
#import the class
from QQuantLib.AE.maximum_likelihood_ae import MLAE

To create an object from the **MLAE** class, the following mandatory arguments must be provided:

1. `oracle`: A QLM `AbstractGate` or `QRoutine` that implements the oracle required for constructing the Grover operator.
2. `target`: The marked state in its binary representation, specified as a Python list.
3. `index`: A list of qubits that will be affected by the Grover operator.

The `MLAE` class internally creates the Grover operator using the `grover` function from the module **QQuantLib/AA/amplitude_amplification**. Consequently, the arguments required for the `MLAE` class are similar to those of the `grover` function (for a detailed explanation, see the notebook **02_AmplitudeAmplification_Operators.ipynb**).

Additionally, an optional dictionary can be provided to configure the algorithm further. The keys for this dictionary include:

- `qpu`: Specifies the QLM solver to be used. If not provided, the default solver will be used.
- `delta`: A float value to set the tolerance threshold for avoiding division-by-zero warnings during computations.
- `optimizer`: The optimizer to be used for solving the optimization problem within the MLAE algorithm (see below for more details).
- `schedule`: A scheduler that determines how the Grover operator is applied throughout the MLAE algorithm (see below for more details).
- `mcz_qlm`: A boolean flag indicating whether to use the QLM multi-controlled Z gate (`True`, default) or the multiplexor-based implementation (`False`).

These configurations allow for fine-tuning the behavior of the MLAE algorithm to suit specific use cases and computational environments.

### 2.1 Creating object from MLAE class

To demonstrate how the **MLAE** class and the algorithm work, we will define the following amplitude estimation problem:

$$
|\Psi\rangle = \mathcal{A}|0\rangle = \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[ \sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right]. \tag{2}
$$

By comparing equation (2) with equation (1):

$$
\sin(\theta)|\Psi_0\rangle = \frac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}} |1\rangle,
$$

and

$$
\cos(\theta)|\Psi_1\rangle = \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[ \sqrt{0}|0\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right].
$$

In this example, the target state is $ |1\rangle $, whose binary representation is $ [0, 0, 1] $. This binary representation must be passed as a list to the `target` variable. Additionally, we need to specify the list of qubits on which the Grover operator will act. In this case, it is the entire register: $ [0, 1, 2] $.

Thus, the mandatory arguments for creating an instance of the **MLAE** class are:
- `oracle`: A QLM `AbstractGate` or `QRoutine` that implements the oracle.
- `target`: `[0, 0, 1]` (binary representation of the target state $ |1\rangle $).
- `index`: `[0, 1, 2]` (list of qubits affected by the Grover operator).

Furthermore, we will provide the QLM solver via the `qpu` key in an input Python dictionary to configure the algorithm.

In [ ]:
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True
}
target = [0,0,1]
index = [0,1,2]
mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

### 2.2 Application of the grover operator

The foundation of any amplitude estimation algorithm lies in the Grover-like operator $\mathcal{Q}$, which is constructed from the oracle operator $\mathcal{A}$ as follows:

$$
\mathcal{Q}(\mathcal{A}) = \mathcal{A} \left(\hat{I} - 2|0\rangle\langle 0|\right) \mathcal{A}^\dagger \left(\hat{I} - 2|\Psi_0\rangle\langle \Psi_0|\right).
$$

This operator has a specific effect on the state $|\Psi\rangle$, which can be expressed as:

$$
\mathcal{Q}^{m_k} |\Psi\rangle = \mathcal{Q}^{m_k} \mathcal{A} |0\rangle = \sin\left((2m_k + 1)\theta\right)|\Psi_0\rangle + \cos\left((2m_k + 1)\theta\right)|\Psi_1\rangle,
$$

where $m_k$ represents the number of times the Grover-like operator is applied. For a deeper understanding of the Grover operator and the amplitude amplification algorithm, refer to the notebook **02_AmplitudeAmplification_Operators.ipynb**.

The process of generating the corresponding Grover operator is handled automatically within the **MLAE** class when an instance is created. To observe the Grover operator in action, you can use the method `run_step` of the class. The inputs for this method are:

- `m_k`: The number of times the Grover-like operator will be applied.
- `n_k`: The number of measurement shots.

The method returns:

- `h_k`: The number of positive outcomes (i.e., the number of times the target state $|\Psi_0\rangle$ is measured).

This functionality allows for the iterative application of the Grover operator, enabling the estimation of the desired amplitude through repeated measurements and statistical analysis.

In [ ]:
m_k = 3
n_k = 100
h_k, circuit = mlae.run_step(m_k,n_k)
print('h_k: '+str(h_k))

In [ ]:
%qatdisplay circuit --depth 0 --svg

From the number of positive outcomes, it is straightforward to estimate the probability of obtaining the state $ |1\rangle $ (the positive outcome):

$$
\sin^2\left((2m_k + 1)\hat{\theta}\right) = \frac{h_k}{n_k} = \hat{p}(|\Psi_0\rangle) \approx p(|\Psi_0\rangle).
$$

Here, we use the $\hat{}$ notation to distinguish between estimated values ($\hat{\theta}$, $\hat{p}(|\Psi_0\rangle)$) and their true counterparts ($\theta$, $p(|\Psi_0\rangle)$).

Thus, we obtain an initial estimation of our target amplitude:

In [ ]:
angle_estimation = np.arcsin(np.sqrt(h_k/n_k))/(2*m_k+1)
estimation = np.sin(angle_estimation)**2

In [ ]:
print("First estimation: ",estimation)
print("Exact probability: ",probability[bitfield_to_int(target)])

The question now is, ¿how can we consistently improve this result?

### 2.3 The Likelihood

To enhance our results, we will conduct multiple experiments and combine their information into a single, more accurate result. This process relies on the concept of **likelihood**. But first, what exactly is likelihood?

In general, the **likelihood** $ L(a|b) $ represents the probability of observing $ b $ given $ a $. Mathematically, this can be expressed as:
$$
L(a|b) = p(b|a).
$$

In our specific case, given the result of an experiment $ h_k $, we are interested in determining the probability that a particular angle $ \theta $ generated this result:
$$
p(\theta|h_k).
$$

From all possible values of $ \theta $, we propose the one with the highest probability as our solution. However, computing $ p(\theta|h_k) $ directly is not straightforward. Instead, we compute $ p(h_k|\theta) $, which is proportional to $ p(\theta|h_k) $ by Bayes' theorem. The associated likelihood is defined as:
$$
L(\theta|h_k) = p(h_k|\theta).
$$

The value of this likelihood is given by:
$$
L(\theta|h_k) = \sin^2\left((2m_k + 1)\theta\right)^{h_k} \cos^2\left((2m_k + 1)\theta\right)^{n_k - h_k}.
$$

This formula arises because each measurement is independent, the probability of obtaining the state $ |1\rangle $ is $ \sin^2\left((2m_k + 1)\theta\right) $, and the probability of obtaining the state $ |0\rangle $ is $ \cos^2\left((2m_k + 1)\theta\right) $.

To compute the likelihood for a given experiment, you can use the function `likelihood`. The inputs for this function are:
- `angle`: The angle $ \theta $ for which the likelihood is being computed.
- `m_k`: The number of times the Grover-like operator is applied.
- `n_k`: The total number of measurement shots.
- `h_k`: The number of positive outcomes (i.e., the number of times the target state $ |1\rangle $ is measured).

The function returns:
- `l_k`: The likelihood of the given angle $ \theta $.

In [ ]:
print('m_k: '+str(m_k)+' n_k: '+str(n_k)+' h_k: '+str(h_k))
theta = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,100)
l_k = np.zeros(len(theta))
for i in range(len(theta)):
    l_k[i] = mlae.likelihood(theta[i],m_k,n_k,h_k)

In [ ]:
plt.plot(theta,l_k, '-', label = r"$L(\theta|h_k)$")
plt.axvline(angle_estimation,color = 'r', label = r"$\hat{\theta}$")
plt.xlabel(r'$\theta$')
plt.ylabel('Likelihood')
plt.grid()
plt.legend()

In blue is depicted the likelihood function for different values of $\theta$. In red we have the estimation we made in the previous section.

### 2.4 The Cost Function 

As we observed in the previous graph, multiple values of $ \theta $ can maximize the likelihood function. However, by combining information from different experiments $(m_k, h_k)$, we can achieve more accurate estimations.

To achieve this, we define a **combined likelihood** as follows:

$$
L(\theta, \mathbf{h}) = \prod_{k=0}^M l_k(\theta, h_k),
$$

where:
$$
\mathbf{h} = (h_0, h_1, \dots, h_M).
$$

Rather than directly solving the maximization problem, we reformulate it as an equivalent minimization problem by introducing the **cost function** $ C(\theta) $, which is defined as:

$$
C(\theta) = -\log\left(L(\theta, \mathbf{h})\right).
$$

The process for computing the **cost function** involves the following steps:

1. **Selecting the schedule of experiments**:
   - First, we need to define the set of experiments $(m_k, h_k)$ using a **schedule**, which is a Python list containing two elements:
     - **1st element**: A list specifying the number of applications of the Grover operator, e.g., $ m_k = [1, 2, 3, 5, 7] $.
     - **2nd element**: A list specifying the number of measurement shots for each corresponding value in $ m_k $, e.g., $ n_k = [100, 200, 50, 50, 100] $.
   - For example, the schedule would be:
     $$
     \text{schedule} = [m_k, n_k] = [[1, 2, 3, 5, 7], [100, 200, 50, 50, 100]].
     $$

2. **Running the experiments**:
   - For each pair $(m_k, n_k)$ in the schedule, execute the `run_step` method to obtain the corresponding number of positive outcomes $ h_k $. This step is automated by the `run_schedule` method of the class.

3. **Computing the cost function**:
   - Using the results $(m_k, n_k, h_k)$ obtained from the experiments, compute the total cost function for different values of $ \theta \in [0, \frac{\pi}{2}] $ using the `cost_function` static method.

This approach allows us to systematically evaluate and minimize the cost function, leading to a more precise estimation of the target amplitude.

#### 2.4.1 Configure the Schedule

As we mentioned earlier, our goal is to combine information from multiple experiments to improve the accuracy of our amplitude estimation. Each experiment can be characterized by two key parameters: 

1. **$m_k$**: The number of applications of the Grover oracle.
2. **$n_k$**: The number of measurement shots.

A collection of these pairs $(m_k, n_k)$ is referred to as a **schedule**. The schedule provides a structured way to define and organize the sequence of experiments.

When creating an instance of the **MLAE** class, the schedule can be specified using the keyword argument `schedule`. This allows the algorithm to systematically execute the defined sequence of experiments and combine their results for more precise estimations.

In [ ]:
m_k = [0,1,2,4,8,16]
n_k = [10]*len(m_k)
schedule = [m_k,n_k]

target = [0,0,1]
index = [0,1,2]

#Schedule not provide
mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': schedule,
    'mcz_qlm': True    
}
mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

print('Schedule: \n', mlae.schedule)


We don't need to define the schedule at initialization. In this case following default schedule will be loaded:

* $m_k$ = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
* $n_k$ = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

Additionally, we can define the schedule after initialization by simply assigning an attribute schedule to the schedule we want

In [ ]:
#Schedule not provide
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True     
}
target = [0,0,1]
index = [0,1,2]
mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

print('Default Schedule: \n', mlae.schedule)

#New Schedule
m_k = [0,1,2,4,8,16]
n_k = [10]*len(m_k)
schedule = [m_k,n_k]
mlae.schedule = schedule

print('New Schedule: \n', mlae.schedule)

#### 2.4.2 run_schedule

Once the schedule is properly configured, we can utilize the `run_schedule` method to execute the application of the Grover operator according to the specified order in the schedule. This method requires the schedule as input and returns an array containing the results of $ h_k $ for each pair $(m_k, n_k)$ defined in the schedule.


In [ ]:
#Creating the class
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True 
}
target = [0,0,1]
index = [0,1,2]
mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

In [ ]:
#Configure the schedule
#applications of Grover operator
m_k = [0, 1, 2, 3, 4, 5]
#Shots for each application of Grover operator
n_k = [100 for i in m_k]
shedule = [m_k, n_k]
#execution of the run_schedule method
h_k = mlae.run_schedule(shedule)
print('h_k: ', h_k)

#### 2.4.3 cost_function

Now that we have all the necessary inputs ($m_k$, $n_k$, $h_k$), we can compute the desired **cost function** $C$. To do this, we use the static method `cost_function` of the **MLAE** class. This method accepts the following parameters:

- `theta`: The angle for which the cost function is being evaluated.
- `m_k`: The number of applications of the Grover operator for each experiment.
- `n_k`: The number of measurement shots for each experiment.
- `h_k`: The number of positive outcomes (i.e., the number of times the target state was measured) for each experiment.

The method computes the **Cost Function** $C(\theta)$ for the given angle $\theta$.

To evaluate the cost function over a range of angles, we can define an array of $\theta$ values between $[0, \frac{\pi}{2}]$ and compute the cost function for each value in this array.

#### Note:
To avoid numerical issues at the boundary angles $0$ and $\frac{\pi}{2}$, we utilize the `delta` property of the **MLAE** class. This ensures that the computations remain stable by introducing a small tolerance around these critical points.

In [ ]:
#Posible angles
angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,50)
cost = np.zeros(len(angles))
#Calculate cost function for each angle
for i in range(len(angles)):
    cost[i] = mlae.cost_function(angles[i], m_k, n_k, h_k)

In [ ]:
#We can plot the Cost Function
plt.plot(angles,cost, '-o', label = r"$C(\theta)$")
plt.title('Cost function')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

We will employ a useful technique by using the `partial` function from the `functools` package to create a new cost function based on the `cost_function` method of the `MLAE` class. In this new cost function, we will fix the values of $m_k$, $n_k$, and $h_k$ that were obtained from the experiments. 

This approach allows us to simplify the computation process: when calling the new function, we only need to provide the angle $\theta$ as a variable input. The previously obtained values of $m_k$, $n_k$, and $h_k$ will act as constants within this new function, ensuring they are consistently used in the calculation of the cost function.

By doing so, we streamline the evaluation of the cost function for different angles $\theta$, making it easier to find the optimal value that minimizes the cost.

In [ ]:
#Little trick: We are going to use the partial from functools for giving the m_k, n_k and h_k
from functools import partial
#Usign partial we create a cost_function using as base the cost_function method from the MLAE class
#were the m_k, n_k and h_k will be fixed to the values we desire. Now this new cost_function only needs
#the angle for return the cost using the m_k, n_k and h_k we provide previously as constant values
partial_cost_function = partial(mlae.cost_function, m_k = m_k, n_k=n_k, h_k=h_k)

In [ ]:
#Possible angles
angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,50)
partial_cost = np.zeros(len(angles))
#Calculate cost function for each angle
for i in range(len(angles)):
    partial_cost[i] = partial_cost_function(angles[i])

In [ ]:
#We can plot the Cost Function
plt.plot(angles,partial_cost, '-o', label = r"$C(\theta)$")
plt.title('Cost function')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

### 2.5 Optimization of the Cost Function

So far, we have defined the **cost function** $ C $ as:

$$
C(\theta) = -\log\left(L(\theta, \mathbf{h})\right),
$$

where $ L(\theta, \mathbf{h}) $ is the combined likelihood function based on the experimental results.

The goal is to find the optimal angle $\theta^*$ that minimizes the cost function:

$$
\theta^* = \arg \min_{\theta} C(\theta).
$$

Once $\theta^*$ is determined, we can compute the desired value of $ a $ using:

$$
a = \sin^2(\theta^*).
$$

This computation can be performed straightforwardly by identifying the minimum value in the `cost_function` array and retrieving the corresponding angle $\theta$. This approach allows us to efficiently estimate the target amplitude $ a $ based on the experimental data.

In [ ]:
#straightforward approach
print('Minimum of the Cost Function: ', min(partial_cost))
theta_min = angles[np.argmin(partial_cost)]
print('theta_min: ', theta_min)

In [ ]:
#Complete plot
plt.plot(angles,partial_cost, '-o', label = r"$C(\theta)$")
plt.axvline(theta_min,color = 'r', label = r"$\theta^*$")
plt.title(r'Cost function and $\theta_{min}$')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

The straightforward approach described earlier provides a good approximation of the optimal angle $\theta^*$, but its accuracy depends on the granularity of the $\theta$ values used to compute the cost function. To achieve a more precise result, a better approach is to employ an optimization routine to find the minimum of the `cost_function`.

One effective method is to use the **brute force optimization** routine from the **SciPy optimization module**. This routine systematically evaluates the cost function over a specified range and step size, ensuring a thorough search for the global minimum. For more details on the `brute` optimization routine, you can refer to the official SciPy documentation:

[SciPy Optimize Brute Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brute.html)

In the following cells, we demonstrate how to apply the `brute` optimization routine to find the minimum of the `cost_function` efficiently.

In [ ]:
import scipy.optimize as so

In [ ]:
#We create a function optimizer that receives as input a function of one variable
theta_domain = [(0+mlae.delta, 0.5*np.pi-mlae.delta)]
optimizer = lambda x: so.brute(func=x, ranges=theta_domain, Ns=1000)

In [ ]:
#We use the optimizer define in before cell given as input the new_cost_function
theta_optimized = optimizer(partial_cost_function)
print('theta_optimized: ', theta_optimized)
print('Cost Function at theta_optimized: ', partial_cost_function(theta_optimized))
print('a estimated from theta_optimized: ', np.sin(theta_optimized)**2)
print("Classical result: ",probability[bitfield_to_int(target)])

In [ ]:
#Now we can plot again all the stuff
plt.plot(angles,partial_cost, '-o', label = r"$C(\theta)$")
plt.axvline(theta_optimized,color = 'r', label = r"$\theta_{optimized}$")
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

### 2.6 Complete Algorithm execution

The **MLAE** class automates the step-by-step process explained in the previous sections through the `mlae` method. Once an instance of the class is created, users can execute the `mlae` method to perform a complete execution of the **MLAE** algorithm.

#### Inputs:
- `schedule`: A predefined list specifying the number of Grover operator applications ($m_k$) and the corresponding number of measurement shots ($n_k$) for each experiment.
- `optimizer`: The optimization routine to be used, which should be passed as a **lambda function**. This function must accept another function (with one variable, the angle to optimize) as its input.

#### Outputs:
- `result`: The result returned by the optimizer, containing details about the optimization process, such as the optimal angle $\theta^*$.
- `h_k`: A list of positive outcomes ($h_k$) obtained from each experiment conducted according to the schedule.
- `cost_function_partial`: A Python function representing the cost function where the values of $m_k$, $n_k$, and $h_k$ are fixed to the results obtained during the experiments.

#### Additional Feature:
This method also updates the `optimizer_time` property of the class, which measures the time taken for the optimization process. This allows users to evaluate the computational efficiency of the algorithm.

By encapsulating all these functionalities into a single method, the `MLAE` class provides a streamlined and user-friendly interface for performing amplitude estimation with maximum likelihood techniques.


In [ ]:
target = [0,0,1]
index = [0,1,2]

#Creating the class
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True     
}

mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)


#Schedule configuration
m_k = [0, 1, 2, 3, 4, 5]
n_k = [100 for i in m_k]
shedule = [m_k, n_k]

#Optimizer Configuration
import scipy.optimize as so
#We create a function optimizer that receives as input the cost_function we create before
theta_domain = [(0+mlae.delta, 0.5*np.pi-mlae.delta)]
optimizer = lambda f: so.brute(func=f, ranges=theta_domain, Ns=1000)

In [ ]:
result, h_k, partial_cost_funtion = mlae.mlae(shedule, optimizer)

In [ ]:
print(result)

In [ ]:
print(h_k)

In [ ]:
print('Optimization time: ', mlae.optimizer_time)

In [ ]:
#We can use the returned partial_cost_function to do some graphs

angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,50)
cost_partial = partial_cost_funtion(angles)
theta_optimized = result[0]

plt.plot(angles, cost_partial, '-o', label = r"$C(\theta)$")
plt.axvline(theta_optimized, color = 'r', label = r"$\theta_{optimized}$")
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

In [ ]:
#Printing the circuit for the oracle
c = mlae._grover_oracle
%qatdisplay c --depth 0 --svg

#### The optimizer

To determine the angle that minimizes the cost function, a minimization algorithm must be employed. When creating an instance of the class, the optimizer can be specified using the `optimizer` key and can later be accessed via the `optimizer` property. By default, the `optimizer` is initialized to the **brute** optimization algorithm from the `scipy.optimize` module.

Additionally, the optimizer can be redefined after initialization by simply assigning a new value to the `optimizer` attribute. This flexibility allows users to choose the most suitable optimization method for their specific problem.

#### Note:
The optimizer provided to the class must be a function of a single variable: the angle $\theta$. This can be easily achieved using **lambda functions**, which allow for concise definition of the required optimization function.

In this example, we will demonstrate the use of the **differential evolution** algorithm from the **SciPy** library as the optimizer. Differential evolution is a global optimization technique that is particularly effective for complex cost functions.

In [ ]:
import scipy.optimize as so

The *differential evolution* algorithm just needs as input the function to be minimized and the bounds where the minimization is done.

In [ ]:
bounds = [[0.+mlae.delta,np.pi/2-mlae.delta]]
differential_evolution = lambda f: so.differential_evolution(f,bounds = bounds)

In [ ]:
result, h_k, new_cost_function = mlae.mlae(schedule, differential_evolution)

Remember that, the result of the function optimize is the result given by the optimizer. In this case, the function differential evolution gives as output an object. The result of the minimization is stored in the attribute *x*.

In [ ]:
print('Optimization time: ', mlae.optimizer_time)

In [ ]:
angle = result.x
print("Quantum result: ",np.sin(angle)**2)
print("Classical result: ",probability[bitfield_to_int(target)])

### BE AWARE!!!

**Brief comment on the optimizer**. As we saw before, the cost function it is not convex, i.e. it has multiple local minima. If we pass a local optimizer such as *Nelder-Mead*, we won't get a good result. Next, we give an example.

In [ ]:
bounds = [[0.+mlae.delta,np.pi/2-mlae.delta]]
#Initial angle value
x0 = [0.8]
nelder_mead = lambda f: so.minimize(f, x0 = x0, bounds = bounds, method = "Nelder-Mead")

In [ ]:
#Remember: if uses other optimizer  than default one the use optimize method instead of run one
mlae.optimizer = nelder_mead
result, h_k, new_cost_function = mlae.mlae(schedule, nelder_mead)

In [ ]:
angles = np.linspace(0++mlae.delta,np.pi/2-mlae.delta,100)
cost = np.zeros(len(angles))
for i in range(len(angles)):
    cost[i] = new_cost_function(angles[i])

In [ ]:
print('Nelder-Mead result: ', result.x)
print('Classical result: ', np.arcsin(probability[bitfield_to_int(target)]**0.5))

In [ ]:
plt.plot(angles,cost,'-o', label = r"$C(\theta)$")
plt.axvline(result.x,color = 'r',label = r"$\theta^*$")
plt.axvline(x0[0],color = 'g',label = r"$\theta_{initial}$")
plt.axvline(angles[cost.argmin()],color = 'b',label = r"$\arg \min_{\theta} C(\theta)$")
plt.title('Nealder-Mead optimization result')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

As can be seen in the graph the *Nelder-Mead* has a bad initialization and optimizer and get stack in a local minimum.

## 3. Summarize and how to use

In section 2 the complete algorithm in a detailed step-by-step was explained. The methods shown in this section were shown only for pedagogical purposes but users **SHOULD NOT** use them.

Users should only use one of the following two methods:

* **mlae** method.
* **run** method.

First, given an Oracle user should create the class:

In Section 2, the complete algorithm was explained in detail through a step-by-step breakdown. The methods described in that section were presented for pedagogical purposes only and are **not intended for direct use** by end users.

For practical applications, users should exclusively utilize one of the following two methods provided by the class:

- **`mlae` method**: In this case the method needs the schedule and optimizer as inputs.
- **`run` method**: This method executes the algorithm using the configuration provided when the `MLAE`class was instantiated..

#### Creating an Instance of the Class:

First, given an Oracle, users should create an instance of the class. This initialization step sets up the necessary components, such as the Oracle operator, target state, and qubit indices, ensuring the algorithm is properly configured for execution.

In [ ]:
target = [0,0,1]
index = [0,1,2]

#Creating the class
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True    
}

mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)


### 3.1 *mlae* method

This method was explained in *Section 2.6*. The user should provide a scheduler and an optimizer. Users can use the defaults in a straightforward way:

In [ ]:
#Default Schedule, default optimizer
result, h_k, partial_cost_function = mlae.mlae(mlae.schedule, mlae.optimizer)

In [ ]:
print("Quantum result theta: ",result)
print("Quantum result a: ",np.sin(result**2))
print('Classical result: ', probability[bitfield_to_int(target)])
print('Test OK: ',abs(np.sin(result**2)-probability[bitfield_to_int(target)]) < 0.005)

In [ ]:
print('Optimizer time: ',mlae.optimizer_time)

### 3.2 *run* method

Additionally, users can configure all the properties of the **MLAE** class and directly invoke the `run` method. This method executes the `mlae` method using the attributes of the **MLAE** class, such as the `schedule` and `optimizer`, and returns the desired value $ a = \sin^2(\theta^*) $. 

Upon execution, the `run` method populates the following properties of the class:

- `h_k`: A list of positive outcomes ($h_k$) corresponding to the schedule used.
- `theta`: The estimated angle $\theta^*$ obtained from a complete execution of the **MLAE** algorithm.
- `ae`: The amplitude estimation parameter extracted from the `theta` property, calculated as $ a = \sin^2(\theta^*) $.
- `partial_cost_function`: A cost function where the variables $m_k$, $n_k$, and $h_k$ are fixed to their respective values from the used schedule. This function accepts only the angle $\theta$ as input.
- `run_time`: The elapsed time for a complete execution of the `run` method.

#### Warning:
The `run` method was specifically designed to work with the **brute** optimization algorithm from the **scipy.optimize** module, which is the default optimizer for the **MLAE** class. If users wish to use alternative optimization algorithms, they **should use the `mlae` method instead** to ensure compatibility and correct functionality.

In [ ]:
target = [0,0,1]
index = [0,1,2]

m_k = [1, 2, 4, 8, 16]
n_k = [100 for i in m_k]

schedule = [m_k, n_k]


#Creating the class
mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': schedule,
    'mcz_qlm': True 
}

mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

estimated_a = mlae.run()

In [ ]:
print('estimated_a: ', estimated_a)
#populated properties
print('mlae.h_k: ', mlae.h_k)
print('mlae.theta: ', mlae.theta)
print('mlae.a: ', mlae.ae)

In [ ]:
angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,100)
#using partial_cost_function attribute

plt.plot(angles, mlae.partial_cost_function(angles), '-o', label = r"$C(\theta)$")
plt.axvline(mlae.theta, color = 'r', label = r"$\theta^*$")
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

In [ ]:
print("Quantum result a: ", estimated_a)
print('Classical result: ', probability[bitfield_to_int(target)])
print('Test OK: ',abs(estimated_a-probability[bitfield_to_int(target)]) < 0.005)

In [ ]:
#For draw the used oracle
grover = mlae._grover_oracle
%qatdisplay grover --depth 3 --svg

Upon executing the `run` method, the following class attributes are populated to provide detailed information about the algorithm's execution:

- `circuit_statistics`: A Python dictionary containing statistics for each quantum circuit used during the algorithm's execution. Each key in the dictionary corresponds to a specific $m_k$ value, and its associated value is another dictionary that holds the complete statistical information of the circuit created for that $m_k$.

- `schedule_pdf`: A pandas DataFrame that stores the complete schedule, including the values of $m_k$ (number of Grover operator applications) and $n_k$ (number of measurement shots), along with the corresponding measurements ($h_k$).

- `oracle_calls`: The total number of oracle calls made during the entire execution of the algorithm.

- `max_oracle_depth`: The maximum number of applications of the oracle throughout the algorithm's execution, representing the deepest level of Grover operator applications.

These attributes provide valuable insights into the performance and resource usage of the algorithm, enabling users to analyze and optimize its behavior effectively.

In [ ]:
mlae.circuit_statistics

In [ ]:
#Complete schedule and the correspondent measurements done
mlae.schedule_pdf

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls for the MLAE was: {}".format(mlae.oracle_calls))

In [ ]:
#Number of maximum oracle applications
mlae.max_oracle_depth

In [ ]:
mlae.quantum_time

In [ ]:
mlae.run_time